# Analyzing CEX Price Impact using CCXT

Here we extract CCXT `trade` and `ohlcv` data to approximate the price impact of trades against centralized exchanges.

In [1]:
import ccxt as ccxt
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import os
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from src.utils.ccxtdatafetcher import CCXTDataFetcher
import asyncio

%load_ext autoreload
%autoreload 2

load_dotenv()
coinbase_pro_api_key = os.getenv("COINBASE_PRO_API_KEY")
coinbase_pro_api_secret = os.getenv("COINBASE_PRO_API_SECRET")
coinbase_pro_api_password = os.getenv("COINBASE_PRO_API_PWD")

In [3]:
datafetcher = CCXTDataFetcher(
    coinbase_pro_api_key, coinbase_pro_api_secret, coinbase_pro_api_password
)

In [44]:
df = pd.read_parquet("../data/trades/binance_ETH_USDC_1699592400.gzip")
cols = [
    "best_price_match",
    "timestamp",
    "aggregate_trade_id",
    "first_trade_id",
    "last_trade_id",
    "maker",
    "price",
    "quantity",
]
df[cols] = df["info"].apply(pd.Series)
df.drop(["info", "fees"], axis=1, inplace=True)
df.drop_duplicates(inplace=True)
print(df.shape)
df.head()

(24874, 17)


,timestamp,symbol,id,order,type,side,takerOrMaker,price,amount,cost,fee,best_price_match,aggregate_trade_id,first_trade_id,last_trade_id,maker,quantity
datetime,,,,,,,,,,,,,,,,,
2023-11-10T00:00:05.141Z,1699574405141,ETH/USDC,30737891,None,None,buy,None,2121.37000000,0.3485,739.297445,None,True,30737891,32082305,32082305,False,0.34850000
2023-11-10T00:00:05.142Z,1699574405142,ETH/USDC,30737892,None,None,buy,None,2121.37000000,2.0084,4260.559508,None,True,30737892,32082306,32082306,False,2.00840000
2023-11-10T00:00:06.595Z,1699574406595,ETH/USDC,30737893,None,None,buy,None,2121.65000000,1.2453,2642.090745,None,True,30737893,32082307,32082307,False,1.24530000
2023-11-10T00:00:06.595Z,1699574406595,ETH/USDC,30737894,None,None,buy,None,2121.65000000,1.1113,2357.789645,None,True,30737894,32082308,32082308,False,1.11130000
2023-11-10T00:00:09.442Z,1699574409442,ETH/USDC,30737895,None,None,buy,None,2121.66000000,0.9629,2042.946414,None,True,30737895,32082309,32082309,False,0.96290000


# Computing Slippage
<img src="../figs/random/binance_trade_description.png" alt="image" width="400"/>

In the Binance data, each trade is a fill. A single trade might be broken up into multiple fills. Let `VWAP` be the volume-weighted average price for a trade, and let `p_{min}` be the minimum price paid for the first fill. The "price impact" paid by the trade can be thought of as `VWAP - p_{min}`, since `p_{min}` was the price at the top of the book when they placed their order, and `VWAP` is what they actually ended up paying (a higher price) for their desired trade size.

To know which trades to group together, we set an arbitrary (but reasonable) time window within which we cluster trades. 

There will be some issues: 

- Fees are not just a function of trade size; each trader might have different fee tiers based on their previous volume, etc.
- 

In [55]:
cols = [
    "best_price_match",
    "timestamp",
    "",
    "first_trade_id",
    "last_trade_id",
    "maker",
    "price",
    "quantity",
]
df.astype({"aggregate_trade_id", int})

TypeError: Cannot interpret '{'aggregate_trade_id', <class 'int'>}' as a data type

In [52]:
df[df["last_trade_id"] == "32082314"]

,timestamp,symbol,id,order,type,side,takerOrMaker,price,amount,cost,fee,best_price_match,aggregate_trade_id,first_trade_id,last_trade_id,maker,quantity
datetime,,,,,,,,,,,,,,,,,
2023-11-10T00:00:20.685Z,1699574420685,ETH/USDC,30737899,None,None,buy,None,2122.62000000,0.1055,223.93641,None,True,30737899,32082313,32082314,False,0.10550000


In [47]:
df[df["last_trade_id"].duplicated(keep=False)]

,timestamp,symbol,id,order,type,side,takerOrMaker,price,amount,cost,fee,best_price_match,aggregate_trade_id,first_trade_id,last_trade_id,maker,quantity
datetime,,,,,,,,,,,,,,,,,


In [24]:
symbol = "ETH/USD"
exchange = datafetcher.coinbasepro
since = exchange.parse8601("2023-11-10 00:00:00")
end = exchange.parse8601("2023-11-11 00:00:00")
cur = int(datetime.now(tz=timezone.utc).timestamp() * 1000)
end = min(end, cur) if end else cur
ohlcv = []

while since < end:
    try:
        new_ohlcv = exchange.fetch_ohlcv(symbol, timeframe="1m", since=since)
        if not len(new_ohlcv):
            break
        since = new_ohlcv[-1][0]
        ohlcv.extend(new_ohlcv)
    except ccxt.RateLimitExceeded:
        exchange.sleep(10000)
    except Exception:
        raise

df = datafetcher.ohlcv_to_df(ohlcv)
df.head()

RequestTimeout: coinbasepro GET https://api.pro.coinbase.com/products/ETH-USD/candles?granularity=60&start=2023-11-10T23%3A06%3A00.000Z&end=2023-11-11T04%3A05%3A00.000Z

In [20]:
datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=timezone.utc)

datetime.datetime(2023, 11, 9, 4, 59, tzinfo=datetime.timezone.utc)

In [9]:
df.tail()

,timestamp,open,high,low,close,volume
timestamp,,,,,,
2023-11-09 04:55:00,1699505700000,1924.01,1924.69,1923.85,1924.59,14.156582
2023-11-09 04:56:00,1699505760000,1924.60,1925.08,1924.32,1924.33,24.231129
2023-11-09 04:57:00,1699505820000,1924.33,1925.45,1924.33,1925.38,11.464312
2023-11-09 04:58:00,1699505880000,1925.43,1926.43,1925.42,1925.70,10.224547
2023-11-09 04:59:00,1699505940000,1925.95,1926.06,1924.31,1924.43,49.804823


In [ ]:
exchange.fetch_order_book(symbol, params={})

# Archived code for using async ccxt

In [ ]:
# This is failing for some reason

exchange = ccxt.coinbasepro(
    {
        "apiKey": coinbase_pro_api_key,
        "secret": coinbase_pro_api_secret,
        "password": coinbase_pro_api_password,
        "enableRateLimit": True,
    }
)

symbol = "ETH/USD"


async def fetch(exchange, page):
    try:
        trades = await exchange.fetch_trades(symbol, params={"after": page})
    except ccxt.RateLimitExceeded as e:
        print("Sleeping")
        await exchange.sleep(10000)
        return await fetch(exchange, page)
    except Exception as e:
        raise e


async def fetch_all(exchange, symbol):
    # Determine how to batch async requests
    trades = await exchange.fetch_trades(symbol)

    # Let's try to do like 10% of the expected pages at a time?
    since = exchange.parse8601("2023-11-09T00:00:00Z")
    expected_range = trades[-1]["timestamp"] - trades[0]["timestamp"]
    requested_range = trades[-1]["timestamp"] - since
    expected_pages = int(requested_range / expected_range)

    batch_size = min(2000, expected_pages)

    first_page = int(exchange.last_response_headers.get("Cb-After"))
    pages = [first_page - 1000 * step for step in range(batch_size)]

    tasks = (asyncio.create_task(fetch(exchange, page)) for page in pages)
    results = await asyncio.gather(*tasks)
    return results


async def main():
    results = await fetch_all(exchange, symbol)
    print(results)
    return results


results = await main()
await exchange.close()